In [1]:
import numpy as np
import pandas as pd

import quandl
import yfinance as yf

In [2]:
info = pd.DataFrame(columns=['Unit', 'Type', 'Description'])
info.loc['DP'] = ['Ratio', 'Index', 'Dividend-Price Ratio of the S&P500']
info.loc['EP'] = ['Ratio', 'Index', 'Earnings-Price Ratio of the S&P500']
info.loc['US10Y'] = ['Yield', 'Index', '10-Year Tnotes']
info.loc['SPY'] = ['Total Return', 'ETF', 'S&P 500']
info.loc['GMWAX'] = ['Total Return', 'Mutual Fund', 'GMO']
info.loc['RF'] = ['Total Return', 'Index', '3-Month Tbills']
info

,Unit,Type,Description
DP,Ratio,Index,Dividend-Price Ratio of the S&P500
EP,Ratio,Index,Earnings-Price Ratio of the S&P500
US10Y,Yield,Index,10-Year Tnotes
SPY,Total Return,ETF,S&P 500
GMWAX,Total Return,Mutual Fund,GMO
RF,Total Return,Index,3-Month Tbills


# Load Data from Quandl

In [3]:
file_key = open("../dev/archive/quandl_key.txt","r")
API_KEY = file_key.read()
file_key.close()

quandl.ApiConfig.api_key = API_KEY

In [4]:
start_date = '1991-10-01'
end_date = '2021-10-31'

sigs_ticks = ["MULTPL/SP500_DIV_YIELD_MONTH","MULTPL/SP500_EARNINGS_YIELD_MONTH","YC/USA10Y"]
sigs_names = ['DP','EP', 'US10Y']
sigs_info = pd.DataFrame({'Name':sigs_names,'Ticker':sigs_ticks}).set_index('Name')

signals = pd.DataFrame()
for idx,tick in enumerate(sigs_info['Ticker']):
    temp = quandl.get(tick, start_date=start_date, end_date=end_date)
    temp.columns = [sigs_info.index[idx]]
    signals = signals.join(temp,rsuffix='_',how='outer')

# some monthly data reported at start of month--assume we do not have it until end of month
signals = signals.resample('M').last()
signals.columns.name = 'SP500 Multiples'
signals

SP500 Multiples,DP,EP,US10Y
Date,,,
1991-10-31,3.17,4.45,7.47
1991-11-30,3.17,4.30,7.38
1991-12-31,3.14,4.11,6.71
1992-01-31,2.94,3.86,7.31
1992-02-29,2.98,3.91,7.27
...,...,...,...
2021-06-30,1.37,2.19,1.45
2021-07-31,1.34,3.64,1.24
2021-08-31,1.32,2.83,1.30


In [5]:
spy_tick = 'EOD/SPY'
data = quandl.get(spy_tick, start_date=start_date, end_date=end_date)[['Adj_Close']]
spy = data.resample('M').last().pct_change()
spy.rename(columns={'Adj_Close':'SPY'},inplace=True)

rf_tick = 'YC/USA3M'
data = quandl.get(rf_tick, start_date=start_date, end_date=end_date)
rf = data.resample('M').last()/(12*100)
rf.rename(columns={'Rate':'US3M'},inplace=True)

gmo_tick = 'GMWAX'
data = yf.download(gmo_tick, start=start_date, end=end_date)['Adj Close']
gmo = data.resample('M').last().pct_change()
gmo.name = gmo_tick
gmo.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [6]:
rets = spy.join(gmo,how='outer')
rets.dropna(axis=0,inplace=True,how='all')
rets

signals, rets = signals.align(rets,join='inner',axis=0)
rf, _ = rf.align(rets,join='inner',axis=0)

# Save Data to Excel

In [7]:
with pd.ExcelWriter('gmo_analysis_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name = 'descriptions')
    signals.to_excel(writer, sheet_name= 'signals')
    rets.to_excel(writer, sheet_name='returns (total)')
    rf.to_excel(writer, sheet_name='risk-free rate')